In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.misc
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import time
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
import cv2

In [2]:
device = torch.device('cuda')

In [3]:
!pwd

/scratch/cz2064/myjupyter/BDML/Project/Phase5/Jupyter_Notebook/Analysis_1


In [4]:
VGG_11 = torch.hub.load('pytorch/vision:v0.6.0', 'vgg11_bn', pretrained=True)
class VGG_CAM(nn.Module):
    def __init__(self, features = VGG_11.features, n_classes = 3):
        super(VGG_CAM, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(512, n_classes,bias=False)
            
    def forward(self, x):
        x = self.features(x)
        self.featuremap1 = x.detach()
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model = VGG_CAM().to(device)
files = torch.load('./Python File/checkpoint_Model.pt', map_location='cpu')
model.load_state_dict(files['best_model_wts'])

Using cache found in /home/cz2064/.cache/torch/hub/pytorch_vision_v0.6.0
/scratch/cz2064/envs/BDML_pip/lib/python3.6/site-packages/torch/serialization.py:623: UserWarning: Couldn't retrieve source code for container of type VGG_CAM. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "


<All keys matched successfully>

In [5]:
test_df_path = '/scratch/cz2064/myjupyter/BDML/Project/Phase5/Train_Test_Split/test.csv'
test_df = pd.read_csv(test_df_path)
test_df.head()

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type,type,label
0,b05abef8-826a-4381-88a4-9920b2df8d3f,TCGA-60-2725-01A-01-TS1.47dfda47-c072-46cb-80d...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-60-2725,TCGA-60-2725-01A,Primary Tumor,Tumor-LUSC,1
1,7dd02309-79ba-450f-8b13-789dbb8fb61a,TCGA-60-2710-11A-01-BS1.570f0c48-7d79-4fe3-927...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-60-2710,TCGA-60-2710-11A,Solid Tissue Normal,Normal,0
2,7aed21df-98a3-49d9-b4b8-c06c432e62bf,TCGA-44-2657-11A-01-TS1.03d328d6-5c0e-42ec-90b...,Biospecimen,Slide Image,TCGA-LUAD,TCGA-44-2657,TCGA-44-2657-11A,Solid Tissue Normal,Normal,0
3,0aebe9e5-eaa9-46dd-ad7b-7d0b06c13b8d,TCGA-66-2780-01A-01-BS1.75a3754a-e4d2-4ef8-ba3...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-66-2780,TCGA-66-2780-01A,Primary Tumor,Tumor-LUSC,1
4,c21f3186-f757-40e9-a576-91ce0a7bfe58,TCGA-94-A5I4-01A-01-TS1.F254FD92-09AE-42D5-AE3...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-94-A5I4,TCGA-94-A5I4-01A,Primary Tumor,Tumor-LUSC,1


In [17]:
sample = test_df.loc[[1]]
sample

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type,type,label
1,7dd02309-79ba-450f-8b13-789dbb8fb61a,TCGA-60-2710-11A-01-BS1.570f0c48-7d79-4fe3-927...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-60-2710,TCGA-60-2710-11A,Solid Tissue Normal,Normal,0


In [18]:
layer2_subimages_path = '/scratch/cz2064/myjupyter/BDML/Project/Data/data_layer2_subimages/'
file_id = sample['File ID'].values[0]
file_name = sample['File Name'].values[0]
sample_label = sample['label'].values[0]
subsamples_file_path = layer2_subimages_path + file_id + '/' + 'subsamples_list.txt'
subsamples_file = open(subsamples_file_path)
subsamples_info = subsamples_file.readlines()
subsamples_file.close()

In [19]:
level_dimensions_info = subsamples_info[0].strip('\n').strip('level_dimensions:')\
.replace('(',' ').replace(')',' ').replace(',',' ').split()
print(level_dimensions_info)
sample_dimensions = (int(level_dimensions_info[2]),int(level_dimensions_info[3]))
print(sample_dimensions)

['40001', '25712', '10000', '6428', '2500', '1607']
(10000, 6428)


In [20]:
pixels = 1024

preprocess = transforms.Compose([
    transforms.CenterCrop(1024),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])


cam_0_dict = {}
cam_1_dict = {}
cam_2_dict = {}
for i in range(int(sample_dimensions[0]/pixels)+1):
    for j in range(int(sample_dimensions[1]/pixels)+1):
        subimage_file_name = 'subsample_'+ str(i) + '_' + str(j) +'.jpg'
        subimage_file_path = layer2_subimages_path + file_id + '/' + subimage_file_name
        image = Image.open(subimage_file_path)
        image_tensor = preprocess(image).unsqueeze(0).to(device,dtype=torch.float)
        model.eval()
        output = model(image_tensor)
        pre_class = output.data.max(-1)[1].cpu().numpy()
        Feature_map = model.featuremap1.cpu().numpy()
        params = list(model.parameters())
        weight_softmax = np.squeeze(params[-1].data.cpu().numpy())
        
        weights_map0 = weight_softmax[0]
        cam_0 = weights_map0.dot(Feature_map.reshape((512,32*32)))
        cam_0 = cam_0.reshape(32,32)
        cam_0 = cv2.resize(cam_0, (1024,1024))
        cam_0_dict[(i,j)] = cam_0
        
        weights_map1 = weight_softmax[1]
        cam_1 = weights_map1.dot(Feature_map.reshape((512,32*32)))
        cam_1 = cam_1.reshape(32,32)
        cam_1 = cv2.resize(cam_1, (1024,1024))
        cam_1_dict[(i,j)] = cam_1
        
        weights_map2 = weight_softmax[2]
        cam_2 = weights_map2.dot(Feature_map.reshape((512,32*32)))
        cam_2 = cam_2.reshape(32,32)
        cam_2 = cv2.resize(cam_2, (1024,1024))
        cam_2_dict[(i,j)] = cam_2
        


In [21]:
CAM_0 = np.zeros(shape=sample_dimensions)
for i in range(int(sample_dimensions[0]/pixels)+1):
    for j in range(int(sample_dimensions[1]/pixels)+1):
        cam_0 = cam_0_dict[(i,j)]
        
        if i == int(sample_dimensions[0]/pixels):
            cam_0 = cam_0[-(sample_dimensions[0]-i*1024):,:]
            i_start = i*1024
            i_end = sample_dimensions[0]
        else:
            i_start = i*1024
            i_end = (i+1)*1024
                
        if j == int(sample_dimensions[1]/pixels):
            cam_0 = cam_0[:,-(sample_dimensions[1]-j*1024):]
            j_start = j*1024
            j_end = sample_dimensions[0]
        else:
            j_start = j*1024
            j_end = (j+1)*1024
        
        CAM_0[i_start:i_end,j_start:j_end] = cam_0

In [22]:
CAM_1 = np.zeros(shape=sample_dimensions)
for i in range(int(sample_dimensions[0]/pixels)+1):
    for j in range(int(sample_dimensions[1]/pixels)+1):
        cam_1 = cam_1_dict[(i,j)]
        
        if i == int(sample_dimensions[0]/pixels):
            cam_1 = cam_1[-(sample_dimensions[0]-i*1024):,:]
            i_start = i*1024
            i_end = sample_dimensions[0]
        else:
            i_start = i*1024
            i_end = (i+1)*1024
                
        if j == int(sample_dimensions[1]/pixels):
            cam_1 = cam_1[:,-(sample_dimensions[1]-j*1024):]
            j_start = j*1024
            j_end = sample_dimensions[0]
        else:
            j_start = j*1024
            j_end = (j+1)*1024
        
        CAM_1[i_start:i_end,j_start:j_end] = cam_1

In [23]:
CAM_2 = np.zeros(shape=sample_dimensions)
for i in range(int(sample_dimensions[0]/pixels)+1):
    for j in range(int(sample_dimensions[1]/pixels)+1):
        cam_2 = cam_2_dict[(i,j)]
        
        if i == int(sample_dimensions[0]/pixels):
            cam_2 = cam_2[-(sample_dimensions[0]-i*1024):,:]
            i_start = i*1024
            i_end = sample_dimensions[0]
        else:
            i_start = i*1024
            i_end = (i+1)*1024
                
        if j == int(sample_dimensions[1]/pixels):
            cam_2 = cam_2[:,-(sample_dimensions[1]-j*1024):]
            j_start = j*1024
            j_end = sample_dimensions[0]
        else:
            j_start = j*1024
            j_end = (j+1)*1024
        
        CAM_2[i_start:i_end,j_start:j_end] = cam_2

In [24]:
original_image_path = '/scratch/cz2064/myjupyter/BDML/Project/Data/data_layer2/' \
+ file_id + '/' + file_name[:-4] + '.jpg'
original_image = cv2.imread(original_image_path)
original_image = cv2.resize(original_image, (1024,1024))

In [25]:
cam = CAM_0
cam_img = cam / np.max(cam)
cam_img = np.uint8(255 * cam_img)
CAM = cv2.resize(cam_img, (1024,1024))
heatmap = cv2.applyColorMap(CAM, cv2.COLORMAP_JET)
cv2.imwrite('cam_0.jpg', heatmap*0.3+0.7*original_image)

True

In [26]:
cam = CAM_1
cam_img = cam / np.max(cam)
cam_img = np.uint8(255 * cam_img)
CAM = cv2.resize(cam_img, (1024,1024))
heatmap = cv2.applyColorMap(CAM, cv2.COLORMAP_JET)
cv2.imwrite('cam_1.jpg', heatmap*0.3+0.7*original_image)

True

In [27]:
cam = CAM_2
cam_img = cam / np.max(cam)
cam_img = np.uint8(255 * cam_img)
CAM = cv2.resize(cam_img, (1024,1024))
heatmap = cv2.applyColorMap(CAM, cv2.COLORMAP_JET)
cv2.imwrite('cam_2.jpg', heatmap*0.3+0.7*original_image)

True